In [15]:
# outputs_path = "/home/zcwang/TRACE/outputs/cl/EPI/TinyLlama/TinyLlama-1.1B-Chat-v1.0/predictions"
# outputs_path = "/home/zcwang/TRACE/outputs/cl/O-LoRA/google/gemma-2b-it/predictions"
# outputs_path = "/home/zcwang/TRACE/outputs/cl/EPI/Qwen/Qwen1.5-0.5B-Chat/predictions"
outputs_path = "/home/zcwang/TRACE/outputs/FSICL/Qwen/Qwen1.5-1.8B-Chat/predictions"

dataset_list = "C-STANCE,FOMC,Py150,ScienceQA,NumGLUE-cm,NumGLUE-ds,20Minuten".split(",")
metric_list = "accuracy,accuracy,similarity,accuracy,accuracy,accuracy,sari".split(",")
scale_list = [100, 100, 1, 100, 100, 100, 1]

import os
import json

task_count = len(dataset_list)
score_matrix = [[] for _ in range(task_count)]
for t in range(task_count):
    for i in range(t+1):
        file_path = os.path.join(outputs_path, f"results-{t}-{i}-{dataset_list[i]}.json")
        data = json.load(open(file_path))

    # if "Qwen" in outputs_path:
        if dataset_list[i] == "FOMC":
            results = data["results"]
            labels = data["labels"]
            results = [res.strip() for res in results]
            # print(results, labels)
            acc = sum([1 for res, label in zip(results, labels) if res == label]) / len(labels)
            # print(f"Accuracy for {file_path}: {acc:.2f}")
            score_matrix[i].append(acc)
            continue

        elif dataset_list[i] == "ScienceQA":
            results = data["results"]
            labels = data["labels"]
            results = [res.strip() for res in results]
            labels = [label[0] for label in labels]
            results = [res[0] if len(res) > 0 else 'A' for res in results]
            # print(results, labels)
            acc = sum([1 for res, label in zip(results, labels) if res == label]) / len(labels)
            # print(f"Accuracy for {file_path}: {acc:.2f}")
            score_matrix[i].append(acc)
            continue

        if metric_list[i] != "sari":
            score_matrix[i].append(data["eval"][metric_list[i]])
        else:
            score_matrix[i].append(data["eval"]["sari"][0]["sari"])

# display the score matrix
print("Score matrix:")
for idx, row in enumerate(score_matrix):
    # expand the row to the full length
    row = ["-"] * (task_count - len(row)) + row + [row[-1] - row[0]]
    print(dataset_list[idx] + ' & ' + ' & '.join([f"{score*scale_list[idx]:.2f}" if not isinstance(score, str) else score for score in row]) + " \\\\")

Score matrix:
C-STANCE & 45.00 & 44.00 & 51.00 & 59.00 & 54.00 & 50.00 & 53.00 & 8.00 \\
FOMC & - & 0.00 & 0.00 & 0.00 & 20.00 & 17.00 & 32.00 & 32.00 \\
Py150 & - & - & 27.22 & 29.58 & 34.61 & 33.03 & 29.48 & 2.26 \\
ScienceQA & - & - & - & 49.00 & 73.00 & 71.00 & 69.00 & 20.00 \\
NumGLUE-cm & - & - & - & - & 9.88 & 11.11 & 12.35 & 2.47 \\
NumGLUE-ds & - & - & - & - & - & 37.00 & 36.00 & -1.00 \\
20Minuten & - & - & - & - & - & - & 38.68 & 0.00 \\


In [16]:
bwf = [(score_matrix[i][-1] - score_matrix[i][0]) * scale_list[i] for i in range(task_count)]
cl = [score_matrix[i][-1] * scale_list[i]for i in range(task_count)]
print(sum(cl) / task_count, sum(bwf) / (task_count - 1))
print(bwf)
print(cl)

38.644261864080995 10.621522633744858
[8.000000000000002, 32.0, 2.2600000000000016, 19.999999999999996, 2.4691358024691357, -1.0000000000000009, 0.0]
[53.0, 32.0, 29.48, 69.0, 12.345679012345679, 36.0, 38.68415403622127]
